In [10]:
import os
import csv
import shutil
from keras.preprocessing.image import ImageDataGenerator
from keras import layers
from keras import models



In [16]:
convolutionalModel = models.Sequential()

convolutionalModel.add( layers.Conv2D( 32, (3, 3), 
                        activation = 'relu', input_shape = (150, 150, 3) ) )

convolutionalModel.add( layers.MaxPooling2D( (2, 2) ) )

convolutionalModel.add( layers.Conv2D( 64, (3, 3), activation = 'relu' ) )

convolutionalModel.add( layers.MaxPooling2D( (2, 2) ) )

convolutionalModel.add( layers.Conv2D( 128, (3, 3), activation = 'relu' ) )

convolutionalModel.add( layers.MaxPooling2D( (2, 2) ) )

convolutionalModel.add( layers.Flatten() )

convolutionalModel.add( layers.Dense( 512, activation = 'relu' ) )

convolutionalModel.add( layers.Dense( 4, activation = 'sigmoid' ) )

convolutionalModel.summary()





_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36992)             0         
__________